In [ ]:
SSH_KEY_LOCATION = "D:\ArkhoTech\CCHC\pem.pem"

from sshtunnel import SSHTunnelForwarder
import datetime as dt 
import pandas as pd
import awswrangler as wr
import pyodbc
import os

HOST = os.getenv("HOST")
USER = os.getenv("USER")
PORT = int(os.getenv("PORT"))
PASSWORD = os.getenv("PASSWORD")

def getConnection(port,password):
    con_string = f"""DRIVER={{ODBC Driver 17 for SQL Server}};SERVER=127.0.0.1,{port};DATABASE=bne;UID=admin;PWD={password}"""
    conn = pyodbc.connect(con_string)
    return conn

def query(query_string: str,password):
    with SSHTunnelForwarder(
            (HOST, PORT),
            ssh_username=USER,
            ssh_pkey=SSH_KEY_LOCATION,
            remote_bind_address=("bne.cppgzemeawv9.us-east-1.rds.amazonaws.com", 1433),
        ) as server:
            conn = getConnection(server.local_bind_port,password)
            return pd.read_sql(query_string, con=conn)

HOST


Identificar los oficios que están buscando las empresas gracias a que existe registro de los CV descargados y los ID de personas. Cada persona puede registrar hasta 3 ocupaciones, pero la mayoría entiendo solo registra 1 oficio. Entonces, identificando qué CV es descargados, podemos identificar el oficio(s) buscado por la empresa.

In [ ]:
df = query("""
select
       convert(varchar(10), F.FEC_ALTA, 120) AS fecha_alta,
       BNE_COMUNAS.NOMBRE                    AS comuna,
       BNE_REGIONES.NOMBRE                   AS region,
       COUNT(*)                              AS total,
        co.DESCRIPCION as ocupacion
from EMP_EMPRESARIO_USUARIOS a
         inner join EMP_USUARIOS_UBICAC b on b.ID_EMPRESARIO_USUARIOS = a.id
         inner join EMP_UBICAC_RELAC c on c.id = b.id_ubic_rel
         inner join emp_empresario d on d.id = c.id_empresario
         inner join sct_emp_sectores e on e.ID_EMPRESARIO = d.id
         inner JOIN dbo.CCHC_DESCARGA_CV F ON a.id = f.id_emp_usuario
         LEFT JOIN BNE_COMUNAS ON BNE_COMUNAS.ID = d.ID_COMUNA
         LEFT JOIN BNE_REGIONES ON BNE_REGIONES.ID = d.ID_REGION
        LEFT JOIN CCHC_DESCARGA_CV_DETALLE CDCD ON CDCD.ID_DESCARGA_CV = F.ID
        LEFT JOIN PER_PERSONAS pp ON pp.id = CDCD.ID_PERSONA
        LEFT JOIN CCHC_PER_OCUPACIONES cpo ON cpo.ID_PERSONA = pp.ID
        LEFT JOIN CCHC_OCUPACIONES co ON co.id = cpo.ID_OCUPACION
        where e.ID_SECTOR = 1 
GROUP BY convert(varchar(10), F.FEC_ALTA, 120), BNE_COMUNAS.NOMBRE, BNE_REGIONES.NOMBRE, DESCRIPCION;
""",PASSWORD)

df.head()

In [ ]:
df["fecha_alta"] = pd.to_datetime(df.fecha_alta)

df.head()

In [ ]:
wr.s3.to_parquet(
    df=df,
    database="raw_dev",
    table="bne_oficios_cvs_descargados",
    path="s3://cchc-dw-dev-raw/bne/oficios_cvs_descargados/",
    mode="overwrite",
    dtype={
        "fecha_alta" : "date"
    },
    dataset=True
)

Contabilizar cuántas personas suben datos de educación superior, de EMTP, de cursos de capacitación.

In [ ]:
# Punto 2

df = query(
    """
SELECT
    CONCAT(per.NUM_DOCUMENTO, per.DIGITO_VERIFICADOR) as rut,
    regiones.NOMBRE as region,
    comunas.NOMBRE as comuna,
    COUNT(media.ID  ) as ed_media,
    COUNT(cursos.ID) as cursos,
    COUNT(superior.ID ) as ed_superior
FROM dbo.PER_PERSONAS per
LEFT JOIN dbo.CCHC_PERSONA_EMTP media ON media.ID_PERSONA = per.ID
LEFT JOIN dbo.CCHC_PERSONA_CURSO cursos ON cursos.ID_PERSONA = per.ID
LEFT JOIN dbo.CCHC_PERSONA_ETESUP superior ON superior.ID_PERSONA = per.ID
LEFT JOIN dbo.BNE_REGIONES regiones ON regiones.id = per.ID_REGION
LEFT JOIN dbo.BNE_COMUNAS comunas ON comunas.id = per.ID_COMUNA
GROUP BY CONCAT(per.NUM_DOCUMENTO, per.DIGITO_VERIFICADOR), regiones.NOMBRE, comunas.NOMBRE
HAVING COUNT(media.ID  ) > 0
    OR COUNT(cursos.ID) > 0
    OR COUNT(superior.ID ) > 0;
""", PASSWORD
)

df.head()


In [ ]:
geografia = wr.s3.read_parquet("s3://cchc-dw-qa-analytics/geografia_variaciones/",columns=['region','comuna'])
geografia

In [ ]:
import unicodedata


def convertir_ascii(campo):
    if campo is None:
        return None

    campo = unicodedata.normalize('NFKD', campo).encode('ascii', 'ignore').decode('UTF-8').lower()
    return campo

def homologar(rut):
    if rut is None:
        return None
    return rut.upper().replace('-','').zfill(10)

dic = {
"Los Lagos":"Los Lagos",
 "Valparaíso":"Valparaíso",
 "Metropolitana":"Metropolitana de Santiago",
 "O'Higgins": "Libertador General Bernardo O'Higgins",
 "Antofagasta":"Antofagasta",
 "Maule":"Maule",
 "La Araucanía":"La Araucanía",
 "Coquimbo":"Coquimbo",
 "Bío Bío":"Biobío",
 "Atacama":"Atacama",
 "Los Ríos":"Los Ríos",
 "Arica y Parinacota":"Arica y Parinacota",
 "Tarapacá":"Tarapacá",
 "Magallanes y Antártica Chilena":"Magallanes y de la Antártica Chilena",
 "Ñuble":"Ñuble",
 "Aysén": "Aysén del General Carlos Ibáñez del Campo"}
 

##staging
df2 = df.copy()

df2.loc[df2.ed_media > 0, "tiene_ed_media"] = True
df2.loc[df2.cursos > 0, "tiene_cursos"] = True
df2.loc[df2.ed_superior > 0, "tiene_ed_superior"] = True

df2.fillna({
    "tiene_ed_media" : False,
    "tiene_cursos" : False,
    "tiene_ed_superior" : False,
}, inplace=True)

###ACA ABAJO ANALYTICS

dfsinComuna = df2[(df2.comuna.isnull()) & (df2.region != 'MIGRACION')]
dfComuna = df2[(df2.comuna.notnull())]

dfComuna.drop(columns=['region'], inplace=True)
dfComuna['comunaAsci'] = dfComuna['comuna'].apply(lambda x: convertir_ascii(x))
geografia['comunaAsci'] = geografia['comuna'].apply(lambda x: convertir_ascii(x))

dfComuna = dfComuna.merge(geografia[['comunaAsci','region']], how='left', on='comunaAsci')


dfsinComuna['region'] = dfsinComuna.region.map(dic)

df2 = pd.concat([dfsinComuna,dfComuna])

#df2['rut'] = df2.rut.apply(lambda x: homologar(x))

df2.drop(columns=["comunaAsci","ed_media", "cursos", "ed_superior"], inplace=True)



df2





In [ ]:
wr.s3.to_parquet(
    df=df2,
    database="raw_dev",
    table="bne_nivel_estudios",
    path="s3://cchc-dw-dev-raw/bne/bne_nivel_estudios/",
    mode="overwrite",
    dataset=True
)

Contabilizar cuántas personas han sido evaluadas en la plataforma por parte de las empresas.

In [ ]:
df = query('''SELECT COUNT(*) as CONTEO, EVALUACION FROM dbo.CCHC_EVALUACION
GROUP BY EVALUACION''',PASSWORD)

df

In [ ]:
wr.s3.to_parquet(
    df=df,
    database="raw_dev",
    table="bne_evaluaciones",
    path="s3://cchc-dw-dev-raw/bne/bne_evaluaciones/",
    mode="overwrite",
    dataset=True
)

???? de aca hasta la proxima etiqueta son pruebas

In [ ]:
df = query('''SELECT RUT,
       LISTA                             AS lista,
       id_usuario,
       NOM_USUARIO,
       numero                            AS persona_lista,
       COALESCE(SUM(ISNULL(NUM_CV, 0)), 0) AS numero_cv_descargados

FROM (SELECT d.RUT, a.id as id_usuario ,aa.ID AS LISTA, a.NOM_USUARIO, F.NUM_CV, g.numero
      FROM dbo.CCHC_LISTAS AS aa
               LEFT OUTER JOIN dbo.EMP_EMPRESARIO_USUARIOS AS a ON aa.ID_EMP_USUARIO = a.ID
               JOIN dbo.EMP_USUARIOS_UBICAC AS b ON b.ID_EMPRESARIO_USUARIOS = a.ID
               INNER JOIN dbo.EMP_UBICAC_RELAC AS c ON c.ID = b.ID_UBIC_REL
               INNER JOIN dbo.EMP_EMPRESARIO AS d ON d.ID = c.ID_EMPRESARIO
               INNER JOIN dbo.SCT_EMP_SECTORES AS e ON e.ID_EMPRESARIO = d.ID
               LEFT OUTER JOIN dbo.CCHC_DESCARGA_CV AS F ON F.ID_LISTA = aa.ID
               LEFT OUTER JOIN (SELECT ID_LISTA, COUNT(*) AS numero
                                FROM dbo.SCT_INVITACIONES AS gg
                                GROUP BY ID_LISTA) AS g ON g.ID_LISTA = aa.ID) AS TMP
GROUP BY RUT, LISTA, numero, NOM_USUARIO, id_usuario''',PASSWORD)

df

In [ ]:
historia_invitaciones = query("""
SELECT
    count(si.ID) as invitaciones,
    cl.ID_EMP_USUARIO as id_usuario,
    datepart(year, FEC_INVITACION) * 1000 + datepart(month, si.FEC_INVITACION) as periodo
FROM dbo.SCT_INVITACIONES si
JOIN dbo.CCHC_LISTAS cl ON si.ID_LISTA = cl.ID
GROUP BY cl.ID_EMP_USUARIO, datepart(year, FEC_INVITACION) * 1000 + datepart(month, si.FEC_INVITACION)
""",PASSWORD)

historia_cvs = query("""
SELECT
    ID_EMP_USUARIO as id_usuario,
    datepart(year, FEC_ALTA) * 1000 + datepart(month, FEC_ALTA) as periodo,
    COUNT(*) as cvs_descargados
FROM dbo.CCHC_DESCARGA_CV
group by ID_EMP_USUARIO, datepart(year, FEC_ALTA) * 1000 + datepart(month, FEC_ALTA)
""",PASSWORD)

creacion_listas = query("""
SELECT
    ID_EMP_USUARIO as id_usuario,
    COUNT(ID) as listas_creadas,
    datepart(year, FEC_ALTA) * 1000 + datepart(month, FEC_ALTA) as periodo
FROM dbo.CCHC_LISTAS
GROUP BY ID_EMP_USUARIO, datepart(year, FEC_ALTA) * 1000 + datepart(month, FEC_ALTA)

""",PASSWORD)

modif_listas = query("""
SELECT
    ID_EMP_USUARIO as id_usuario,
    COUNT(ID) as listas_modificadas,
    datepart(year, FEC_MODIF) * 1000 + datepart(month, FEC_MODIF) as periodo
FROM dbo.CCHC_LISTAS
GROUP BY ID_EMP_USUARIO, datepart(year, FEC_MODIF) * 1000 + datepart(month, FEC_MODIF)

""",PASSWORD)


personas_empresas = query("""
SELECT
    eeu.RUT rut_usuario,
    eeu.id as id_usuario,
    empresa.id as id_empresa,
    empresa.rut as rut_empresa
FROM dbo.EMP_EMPRESARIO_USUARIOS eeu
         JOIN dbo.EMP_USUARIOS_UBICAC ubicac ON ubicac.ID_EMPRESARIO_USUARIOS = eeu.ID
         JOIN dbo.EMP_UBICAC_RELAC eur ON eur.ID = ubicac.ID_UBIC_REL
         JOIN dbo.EMP_EMPRESARIO empresa ON empresa.ID = eur.ID_EMPRESARIO
        JOIN dbo.SCT_EMP_SECTORES sector_empresa ON sector_empresa.ID_EMPRESARIO = empresa.ID
        JOIN dbo.SCT_SECTORES sector ON sector.id = sector_empresa.ID_SECTOR
""",PASSWORD)

In [ ]:
from datetime import date, datetime
def generate_base_sabana():
    """Genera un dataframe con los campos year, month y fecha (datetime) desde el 1980 hasta la fecha actual

    Returns:
        pd.DataFrame: Dataframe para utilizar como maestro tiempo
    """
    data = []
    for y in range(2020, datetime.now().year + 1):
        for m in range(1, 13):
            data.append({"year": y, "month": m, "fecha": date(y, m, 1), "periodo" : y * 1000 + m})
    df = pd.DataFrame(data)
    df.fecha = pd.to_datetime(df.fecha)
    return df

df = generate_base_sabana()

In [ ]:
cvs = df.merge(
    historia_cvs,
    "left",
    "periodo"
)

cvs = cvs.merge(
    historia_invitaciones,
    "left",
    ["periodo", "id_usuario"]
)

cvs = cvs.merge(
    creacion_listas,
    "left",
    ["periodo", "id_usuario"]
)

cvs = cvs.merge(
    modif_listas,
    "left",
    ["periodo", "id_usuario"]
)


cvs.loc[(cvs.cvs_descargados > 0) | (cvs.invitaciones > 0) | (cvs.listas_creadas > 0) | (cvs.listas_modificadas > 0) , "is_activo"] = True
##HACER REFERENCIA AL TIEMPO 

cvs = cvs[cvs.id_usuario.notnull()]

cvs = cvs.merge(
    personas_empresas[["id_usuario", "rut_empresa"]]
    .drop_duplicates(),
    "left",
    "id_usuario"
)


#cvs[cvs.is_activo == True].groupby(["periodo"], as_index=False).agg({"ID_EMP_USUARIO" : "count"})
cvs = cvs[cvs.is_activo == True]
cvs.id_usuario.to_list()
# modif_listas.merge(
#     creacion_listas,
#     "cross",
#     ["ID_EMP_USUARIO", "periodo"]
# )

In [ ]:
wr.s3.to_parquet(
    df=df2,
    database="raw_dev",
    table="bne_usuarios_empresas",
    path="s3://cchc-dw-dev-raw/bne/bne_usuarios_empresas/",
    mode="overwrite",
    dataset=True
)

BNE_USUARIOS_EMPRESAS FINAL QUERY + WRITER
Codigo abajo


In [ ]:
df = query("""
SELECT d.RUT,
       a.ID as id_usuario,
       nom_usuario,
       convert(varchar(10), F.FEC_ALTA, 105) AS FECHA_ALTA,
       F.ID_LISTA ,
       COUNT(*) AS TOTAL,
       SUM(F.NUM_CV) as DESCARGA_CV
FROM EMP_EMPRESARIO_USUARIOS a
INNER JOIN EMP_USUARIOS_UBICAC b ON b.ID_EMPRESARIO_USUARIOS=a.id
INNER JOIN EMP_UBICAC_RELAC c ON c.id=b.id_ubic_rel
INNER JOIN emp_empresario d ON d.id=c.id_empresario
INNER JOIN sct_emp_sectores e ON e.ID_EMPRESARIO=d.id
INNER JOIN dbo.CCHC_DESCARGA_CV F ON a.id=f.id_emp_usuario
where e.ID_SECTOR = 1
GROUP BY d.RUT,
         a.ID ,
         nom_usuario,
         convert(varchar(10), F.FEC_ALTA, 105),
        f.ID_LISTA ;""",PASSWORD)
len(df)

In [ ]:
def dateFormat(date):
    date = date.split("-")
    date = "01-{}-{}".format(date[1],date[2])
    return date

def date(date):
    date = date.split("-")
    date = "{}-{}-{}".format(date[0],date[1],date[2])
    return date

In [ ]:

df2 = df.copy()
no_considerar = wr.s3.read_parquet(path='s3://cchc-dw-qa-staging/bne/empresas_no_considerar/')
no_considerar.rename(columns={'rut':'RUT'},inplace=True)
no_considerar['ignorar'] = True

empresa = wr.s3.read_parquet(path='s3://cchc-dw-qa-raw/bne/empresas/year=2023/month=02/',columns=['RUT','ESTADO'])
empresa = empresa[empresa.ESTADO == 1]


df2.FECHA_ALTA = df2.FECHA_ALTA.apply(lambda x: date(x))
df2["FECHA_ALTA_MES"] = df2.FECHA_ALTA.apply(lambda x: dateFormat(x))


df2.FECHA_ALTA = pd.to_datetime(df2.FECHA_ALTA, format='%d-%m-%Y')
df2["FECHA_ALTA_MES"] = pd.to_datetime(df2.FECHA_ALTA_MES, format='%d-%m-%Y')

df2 = df2.merge(empresa, 'left', 'RUT')
df2 = df2.merge(no_considerar, 'left', 'RUT')

df2 = df2[df2.ignorar != True]

df2 = df2[['RUT','id_usuario','nom_usuario','FECHA_ALTA','FECHA_ALTA_MES','ID_LISTA','TOTAL','DESCARGA_CV']]

df2.rename(columns={"TOTAL":"VISTAS_CV"},inplace=True)

df2


In [ ]:
wr.s3.to_parquet(
    df=df2,
    database="raw_dev",
    table="bne_usuarios_empresas_v2",
    path="s3://cchc-dw-dev-raw/bne/bne_usuarios_empresas_v2/",
    mode="overwrite",
    dataset=True
)

pruebas

In [ ]:
df = wr.s3.read_parquet(path='s3://cchc-dw-qa-raw/bne/cv_vistos/year=2023/month=02/')

len(df)


In [ ]:
df = query("""SELECT d.rut,
         nom_usuario,
         CONVERT(VARCHAR(10), F.fec_alta, 120) AS FECHA_ALTA,
         Count(*)                              AS TOTAL
  FROM   emp_empresario_usuarios a
         INNER JOIN emp_usuarios_ubicac b
                 ON b.id_empresario_usuarios = a.id
         INNER JOIN emp_ubicac_relac c
                 ON c.id = b.id_ubic_rel
         INNER JOIN emp_empresario d
                 ON d.id = c.id_empresario
         INNER JOIN sct_emp_sectores e
                 ON e.id_empresario = d.id
         INNER JOIN dbo.cchc_descarga_cv F
                 ON a.id = f.id_emp_usuario
        WHERE e.ID_SECTOR = 1
  GROUP  BY d.rut,
            nom_usuario,
            CONVERT(VARCHAR(10), F.fec_alta, 120); """,PASSWORD)

aux = df.TOTAL.sum()

aux

tabla que usaremos para la dim

SUM dim_bne_empresas[cv_vistos],
where dim_bne_empresas[USO] = TRUE

In [ ]:
#intento migracion spark to pandas
staging = 's3://cchc-dw-qa-staging/'
analytics = 's3://cchc-dw-qa-analytics/'
empresas_df2 = wr.s3.read_parquet(staging + "bne/empresas")
actividad_emps_df2 = wr.s3.read_parquet(staging + "bne/actividad_empresas")
ignorar2 = wr.s3.read_parquet(staging + "bne/empresas_no_considerar")
cvs_vistos2 = wr.s3.read_parquet(staging + "bne/cv_vistos")
dim_socios2 = wr.s3.read_parquet(analytics + "dim_socios")
nomina_empresas2 = wr.s3.read_parquet(
    staging + "sii/nomina_empresas/commercial_year=2021/"
)

In [ ]:
empresas_df = empresas_df2.copy()
actividad_emps_df = actividad_emps_df2.copy()
ignorar = ignorar2.copy()
cvs_vistos = cvs_vistos2.copy()
dim_socios = dim_socios2.copy()
nomina_empresas = nomina_empresas2.copy()

nomina_empresas = nomina_empresas[["rut", "numero_de_trabajadores_dependientes_informados",
                                  "tamano_de_empresa", "tramo_de_facturacion_txt",
                                  "tramo_trabajadores"]]

# Reemplazo de '-' y transformación a mayúsculas en el RUT
nomina_empresas['rut'] = nomina_empresas['rut'].str.replace("-", "")
nomina_empresas['rut'] = nomina_empresas['rut'].str.upper()

# Filtrado de empresas activas
empresas_df = empresas_df[empresas_df['estado'] == 1]

# Unión con la tabla ignorar
ignorar = ignorar[['rut']]
ignorar['ignorar'] = True
df = pd.merge(empresas_df, ignorar, on='rut', how='left')

actividad_emps_df['numero_cv_descargados'] = actividad_emps_df['numero_cv_descargados'].astype('int64')
actividad_emps_df.fillna({'persona_lista':'0'},inplace=True)
actividad_emps_df['persona_lista'] = actividad_emps_df['persona_lista'].astype('int64')

# Agrupación y suma de actividad_emps_df
cantidad_cvs_ae = actividad_emps_df.groupby("rut").agg({'persona_lista': 'sum',
                                                         'numero_cv_descargados': 'sum'})
cantidad_cvs_ae = cantidad_cvs_ae.rename(columns={'persona_lista': 'persona_lista',
                                                   'numero_cv_descargados': 'numero_cv_descargados'})

# Agrupación y suma de cvs_vistos
cv_agg = cvs_vistos.groupby("rut").agg({'total': 'sum'})
cv_agg = cv_agg.rename(columns={'total': 'cv_vistos'})

df = df.merge(cantidad_cvs_ae, on='rut', how='left')
df = df.merge(cv_agg, on='rut', how='left')

socios_unicos = (
    dim_socios.loc[dim_socios['estado'] == 'ACTIVO']
    .drop_duplicates(subset=['rut'])
    .loc[:, ['rut']]
    .assign(socio=1)
    .assign(rut=lambda x: x['rut'].str.replace("-", ""))
)

df = df.merge(socios_unicos, on='rut', how='left')

df.loc[(df['cv_vistos'] > 0) |
    (df['numero_cv_descargados'] > 0) |
    (df['persona_lista'] > 0) &
    (df['estado'] == 1) &
    (df['ignorar'].notnull()), 'uso'] = True


df = df.fillna({
    'persona_lista': 0,
    'numero_cv_descargados': 0,
    'cv_vistos': 0,
    'socio': 0,
    'ignorar': False,
})

df = df[(df.uso == True) & (df.cv_vistos > 0)]

aux = df.cv_vistos.sum()
aux


16538+2982 = 1950
19363 TOTAL PAGINA
3106 DESCARGADOS PAGINA

19687 VISTA BNE


322

In [ ]:
cv = wr.s3.read_parquet(path='s3://cchc-dw-qa-raw/bne/cv_vistos/year=2023/month=02/')
empresa = wr.s3.read_parquet(path='s3://cchc-dw-qa-raw/bne/empresas/year=2023/month=02/')
empresa = empresa[empresa.ESTADO == 1]
actividad = wr.s3.read_parquet(path='s3://cchc-dw-qa-raw/bne/actividad_empresas/year=2023/month=02/')

df = cv.merge(empresa,'left','RUT')

no_considerar.rename(columns={"rut":"RUT"},inplace=True)
no_considerar['ignorar'] = True

df = df.merge(no_considerar,'left','RUT')

df = df[df.ignorar != True]

df.TOTAL.sum()
len(cv)

In [ ]:
empresa = wr.s3.read_parquet(path='s3://cchc-dw-qa-raw/bne/empresas/year=2023/month=02/')
actividad = wr.s3.read_parquet(path='s3://cchc-dw-qa-raw/bne/actividad_empresas/year=2023/month=02/')
no_considerar = wr.s3.read_parquet(path='s3://cchc-dw-qa-staging/bne/empresas_no_considerar/')
no_considerar.rename(columns={"rut":"RUT"},inplace=True)
no_considerar['ignorar'] = True

empresa = empresa[empresa.ESTADO == 1]


actividad.numero_cv_descargados = actividad.numero_cv_descargados.astype('int')


df = empresa.merge(actividad,'left','RUT')

df2 = df.merge(no_considerar, 'left', 'RUT')


df2 = df2[df2.ignorar != True]

actividad.numero_cv_descargados.sum()

